## ● 機器學習演算法-DBSCAN 修改為 P-DBSCAN與應用 MingHsun 2016/03/15
* 主要原因:應用DBSCAN於大量的Flickr 照片資料，找尋地理空間上高密度拍照與人群聚集之地區，* 然而在Flickr資料中經常有，一位使用者有多張拍照的問題' ，因而將照片數量作為DBSCAN minpoint參數會形成偏誤，因此自行修改DBSCAN之演算法為P-DBSCAN，以拍照者數量作為minpoint參數。
* 應用:
> 1.讀取Shapefile資料中點位資料<br>
  2.應用修改DBSCAN後的P-DBSCAN於點資料中以找尋clustering<br>
  3.使用convexhull繪製這些clustering的邊界<br>
  4.將這些邊界輸出成Shapefile
  
在本篇中還包含了pygmal與picke的說明與使用

## 1.讀取Shapefile資料中點位資料

In [ ]:
import time
import shapefile
dataset=[]
# y_list=[2001,2002,2003,2004,2005,2006,2007,2008,2009,2010,2011,2012,2013,2014,2015]
y_list=[2012]
for yy in y_list:
    year=str(yy)
    sf = shapefile.Reader("clip/"+year+"clip.shp")
    shapes = sf.shapes()
    fields = sf.fields
    records = sf.records()
    for attribute in records:
        dataset.append((attribute[7],attribute[8],attribute[1],attribute[5],attribute[4]))
print ("原始資料數量",len(dataset))
check_dataset_list=list(set(dataset))
print ("排除同人同地點重複",len(check_dataset_list))

## 2. P-DBSCAN Algorithm
*DBSCAN主要特點為能夠跨越邊界的限制，排除低密度的雜訊點位 (Noise Point) 以找處高密度的群集點位 -要搜尋半徑範圍 (Radius) 與形成核心群集的資料點數量門檻值 (Min-Points)

In [5]:
#參數
D=check_dataset_list
eps=10
min_owner=6

In [ ]:
def regionQuery(p):
     return  (((p[0]-p_x)**2+ (p[1]-p_y)**2)**0.5)<=eps
    
# def ownercheck(regionQ_list):
#     check_list=regionQ_list
#     neighborID_list=[]
#     for pp in check_list:
#         if pp[2] not in neighborID_list:
#             neighborID_list.append(pp[2])
#     return neighborID_list
def ownercheck(p):
    neighbor_ID_list.append(p[2])

In [3]:
noise=[]
visited=[]
cluster={}
cluster_count=-1
f=time.time()
print (f)
for point in D:
    if point not in visited:
        visited.append(point)
        p_x=point[0]
        p_y=point[1]
        neighbor_plist = filter(regionQuery,D)
        neighbor_ID_list=[]
        map(ownercheck,neighbor_plist)
        neighbor_num=len(set(neighbor_ID_list))
        if neighbor_num < min_owner:
            noise.append(point)
        else:
            cluster_count=cluster_count+1
            name = 'cluster'+str(cluster_count);
            cluster.setdefault(name,[])
            cluster[name].append(point)
            
            for p in neighbor_plist:
                if p not in visited:
                    visited.append(p)
                    if p not in cluster[name]:
                        cluster[name].append(p)
                    p_x=p[0]
                    p_y=p[1]
                    n_plist=filter(regionQuery,D)
                    neighbor_ID_list=[]
         
                    map(ownercheck,n_plist)
                    n_num=len(set(neighbor_ID_list))
                    if n_num>=min_owner:
                        
                        sub=set(n_plist)-set(neighbor_plist)
                        sub_list=list(sub)
                        for n in sub_list:
                            neighbor_plist.append(n)

e=time.time()
print (e)
print (e-f)

1499775413.5376687
1499775709.3765898
295.8389210700989


## 3.將P-DBSCAN 分析成果，使用convexhull繪製集群邊界

In [58]:
#統計集群 並且將集群進行append 以方便後續convexhull計算
convex_dict={}
c=0
print ("cluster_number",len(cluster))
for aa in cluster:
    convex_dict.setdefault(aa,[])
    c=c+1
    for bb in cluster[aa]:
#         c=c+1
        convex_dict[aa].append([float(bb[4]),float(bb[3])])
    print (aa,len(cluster[aa]))
# print "all",c
print ("noise",len(noise))

cluster_number 0
noise 59273


In [4]:
#計算convexhull
def vertice_value(v):
    convex_plist.append(tuple(convex_dict[aa][v]))
convex_pdict={}
from scipy.spatial import ConvexHull
for aa in convex_dict:
    convex_plist=[]
    hull=ConvexHull(convex_dict[aa])
    map(vertice_value,hull.vertices)
    convex_plist.append(tuple(convex_plist[0]))
    convex_pdict.setdefault(aa,convex_plist)
# print convex_pdict

## module-pygmaps
* 一個可以客製化繪製地理資料於Google Map的模組<BR>
* 將上述的成果繪製到地圖上呈現

In [ ]:
#畫圖
import pygmaps
import random
lat, lng = 25.05565, 121.56
mymap = pygmaps.maps(lat, lng, 13)
# color_list=[]
#繪製點位
for c in cluster:
    color = "#%06x" % random.randint(0, 0xFFFFFF)
    for cp in cluster[c]:
        y=float(cp[4])
        x=float(cp[3])
        mymap.addpoint(y,x,color,str(c))

all_con_list=[]
#繪製convehull
try:
    for con in convex_pdict:
        conlist=convex_pdict[con]

        all_con_list.append(conlist)
        mymap.addpath(conlist,"#00FF00")
except:
    print (error)
    
# # #繪製noise
# for n in noise:
#     y=float(n[4])
#     x=float(n[3])
#     mymap.addpoint(y,x,"#F6E3CE","noise")

    
mymap.draw('p-dbscan.html')
from IPython.display import HTML
HTML('<iframe src=dbscan.html width=1000 height=2000></iframe>')

## module-pickle
* 除了基本I/O來保存運算結果之外，物件序列化（Serialization)，就是如果想直接保存物件狀態，在下次重新執行程式時讀取以恢復運算時必要的資料，這類的技術稱為物件序列化（Object serialization）。
* Q:目的為為何呢?A:方便存取使用
* 像在P-DBSCAN的這個運算當中，由於資料量龐大且此演算法是一個較耗時的演算法，因此以pickle來存取運算的結果，以方便後續使用。


pickle兩個主要函數
>dump()  進行物件序列化<br>
 load() 載入pickle檔<br>

In [11]:
import pickle
convex_pdict_p= convex_pdict
pickle.dump(convex_pdict_p, open( "convex_pdict_p.p", "wb" ))

In [12]:
print convex_pdict

{'cluster0': [(25.0346, 121.565), (25.033588, 121.565351), (25.033078, 121.564944), (25.032883, 121.564385), (25.033272, 121.563999), (25.033559, 121.563721), (25.034249, 121.563914), (25.034438, 121.564171), (25.0346, 121.565), '#00FF00']}


## 4.GIS-GDAL
*  繪製convexhull to polygon(存放至:cluster_dhp資料夾)/

In [13]:
import os
import osgeo.ogr as ogr
import osgeo.osr as osr
for clust in convex_pdict:
    outSHPfn = "cluster_shp/"+clust+".shp"
    # set up the shapefile driver
    driver = ogr.GetDriverByName("ESRI Shapefile")

    #delete shp
    if os.path.exists(outSHPfn):
        driver.DeleteDataSource(outSHPfn)
    # create the data source
    data_source = driver.CreateDataSource(outSHPfn)
    # create the spatial reference, WGS84
    srs = osr.SpatialReference()
#     srs.ImportFromEPSG(4326)
    srs.SetWellKnownGeogCS( "WGS84" );
#     spatialReference = osr.SpatialReference()
#     spatialReference.ImportFromProj4('+proj=longlat +ellps=WGS84 +datum=WGS84 +no_defs')
    # create the layer
    # layer = data_source.CreateLayer(outSHPfn,srs, ogr.wkbPoint)
    layer = data_source.CreateLayer(outSHPfn,srs, ogr.wkbPolygon)

    # create the feature
    feature = ogr.Feature(layer.GetLayerDefn())

    # # Add the fields we're interested in
    # field_photoID = ogr.FieldDefn("Name", ogr.OFTString)
    # field_photoID.SetWidth(24)
    # layer.CreateField(field_photoID)
    # Create ring
    ring = ogr.Geometry(ogr.wkbLinearRing)
    for i in range(0,len(convex_pdict[clust])-1):
#         print convex_pdict[clust][i][1],convex_pdict[clust][i][0]
        ring.AddPoint(convex_pdict[clust][i][1],convex_pdict[clust][i][0])

    poly = ogr.Geometry(ogr.wkbPolygon)
    poly.AddGeometry(ring)
    # Set the feature geometry using the point
    feature.SetGeometry(poly)
    # Create the feature in the layer (shapefile)
    layer.CreateFeature(feature)
    # Destroy the feature to free resources
    feature.Destroy()
    # Destroy the data source to free resources
    data_source.Destroy()
#     print "-----"